In [ ]:
begin
	import Pkg
	Pkg.activate("..")
	Pkg.instantiate()
	
    using LinearAlgebra
	
    push!(LOAD_PATH, "$(@__DIR__)/../src")
	using NoisyReach
    using model
end

  Activating project at `~/NoisyReach.jl`
Precompiling project...
           ✗ NoisyReach
  0 dependencies successfully precompiled in 5 seconds. 216 already precompiled.
  1 dependency errored.
  For a report of the errors see `julia> err`. To retry use `pkg> precompile`
[ Info: Precompiling NoisyReach [caa4e8f2-8139-4784-bc83-fd977aec17f5] (cache misses: include_dependency fsize change (2))
